In [15]:
# install package
%pip install -U langchain-ollama

  Using cached langchain_ollama-0.2.1-py3-none-any.whl.metadata (1.9 kB)
Using cached langchain_ollama-0.2.1-py3-none-any.whl (15 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
%pip install ollama
%pip install torch
%pip install langchain
%pip install sentencepiece
%pip install deepspeed --upgrade
%pip install accelerate


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


...


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached deepspeed-0.16.1.tar.gz (1.4 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [23 lines of output]
      [WARNING] Unable to import torch, pre-compiling ops will be disabled. Please visit https://pytorch.org/ to see how to properly install torch on your system.
       [WARNING]  unable to import torch, please install it if you want to pre-compile any deepspeed ops.
      DS_BUILD_OPS=1
      Traceback (most recent call last):
        File "d:\hf\code\data_code\text2sql\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "d:\hf\code\data_code\text2sql\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "d:\hf\code\data_code\text2sql\.venv\Lib\site-packages\pip\_vendor\pyp

  Using cached accelerate-1.2.0-py3-none-any.whl.metadata (19 kB)
  Using cached huggingface_hub-0.26.5-py3-none-any.whl.metadata (13 kB)
  Using cached safetensors-0.4.5-cp312-none-win_amd64.whl.metadata (3.9 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
Using cached accelerate-1.2.0-py3-none-any.whl (336 kB)
Using cached huggingface_hub-0.26.5-py3-none-any.whl (447 kB)
Using cached safetensors-0.4.5-cp312-none-win_amd64.whl (286 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [27]:
import os
import re
from langchain.chains import LLMChain
from langchain import PromptTemplate, LLMChain
from langchain.llms import Ollama

# Define your transaction columns as a list
transaction = [
    "transaction_id",
    "transaction_amount",
    "transaction_date",
    "transaction_type",
    "transaction_status",
    "transaction_description",
    "transaction_source_account",
    "transaction_destination_account",
    "transaction_currency",
    "transaction_fee"
]

# Initialize Ollama LLM
llm = Ollama(model="mistral:7b-instruct")

# Function to match columns with the user's question
def find_columns_match(question, input_dict):
    try:
        question_list = re.split(r'\s|,|\.', question)
        for index, string2 in enumerate(question_list):
            for string1 in input_dict:
                score = SequenceMatcher(None, string1.lower(), string2.lower()).ratio() * 100
                if score > 91:
                    question_list[index] = string1 + ","
        return " ".join(question_list)
    except:
        return question

# Function to generate SQL query using Langchain
def query_generator(tble, cols, question):
    template = """Generate a SQL query using the following table name: {Table}, and columns as a list: {Columns}, to answer the following question:
    {question}.
    
    Output Query:
    """
    prompt = PromptTemplate(template=template, input_variables=["Table", "question", "Columns"])
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    
    response = llm_chain.run({"Table": tble, "question": question, "Columns": cols})
    return response

# Define the list of input questions
inputs = [
    "Generate an SQL query to retrieve transaction_amount, transaction_date, transaction_type, transaction_description where transaction_id is 10",
    "Generate an SQL query to retrieve transaction_id, transaction_date, transaction_type, transaction_source_account where transaction_status is 'completed'",
    "Generate an SQL query to retrieve count of the transaction_type and their average transaction_amount, ordered by transaction_type.",
    "Generate an SQL query to retrieve list of the total transaction amount for each source account, sorted by total transaction amount in descending order.",
    "Generate an SQL query to retrieve find the maximum transaction amount for each transaction type, ordered by transaction type."
]

# Execute the query generation for each input
for input in inputs:
    matched_question = find_columns_match(input, transaction)
    result = query_generator("transaction", transaction, matched_question)
    print(result)


 SELECT transaction_amount, transaction_date, transaction_type, transaction_description
   FROM transaction
   WHERE transaction_id = 10;
 SELECT transaction_id, transaction_date, transaction_type, transaction_source_account
   FROM transaction
   WHERE transaction_status = 'completed';
```sql
SELECT transaction_type, COUNT(transaction_id) as total_transactions, AVG(transaction_amount) as average_transaction_amount
FROM transaction
GROUP BY transaction_type
ORDER BY transaction_type;
```
This SQL query retrieves the count of transactions for each `transaction_type` and their corresponding average `transaction_amount`, ordering the result by `transaction_type`.
 Here is the SQL query you requested. This query retrieves the total transaction amount for each source account, sorts the results in descending order by total transaction amount, and groups the results by the source account.

```sql
SELECT transaction_source_account, SUM(transaction_amount) as Total_Transaction_Amount
FROM trans